<a href="https://colab.research.google.com/github/shearere2/Miscellaneous/blob/main/Unpack_Truetime_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import zipfile

def unpack_truetime(path:str) -> pd.DataFrame:
  with zipfile.ZipFile(path, 'r') as zipped_files:
    i = 0
    for file_name in zipped_files.namelist():
      if i == 0:
        df = pd.read_csv(zipped_files.open(file_name)).T # Set main dataset
        df.columns = df.iloc[1].apply(lambda x: str(x)) # Set column names to Route
        df = df[2:] # Remove route row (repeat of index) and excel index row

      else:
        curr = pd.read_csv(zipped_files.extract(file_name)).T
        curr.columns = curr.iloc[1].apply(lambda x: str(x))
        curr = curr[2:]
        df = pd.concat([df, curr]) # Add current iteration data to main dataset

      i += 1

  df = df.fillna(0)
  df = df.sort_index()
  return df.reset_index().rename(columns={'index':'Collection Time'})
x = unpack_truetime('sat_sun_truetime.zip')

<ipython-input-16-ebcf61d2c8d1>:21: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(0)


The following 2 cells walk through problem-solving process for route to route entity resolution. For these cells to make sense, remove '.apply(lambda x: str(x))' from the curr.columns = and df.columns = lines (thus breaking the code, but making these 2 cells make sense)

In [ ]:
x.columns # Why is there 51 and '51'? int 51 contains basically no information,
# str 51 contains most of information for route 51

Index(['Collection Time', '1', '2', '4', '8', '12', '13', '14', '15', '16',
       '17', '20', '21', '22', '24', '26', '27', '29', '31', '36', '38', '39',
       '40', '41', '43', '44', '48', '51', '54', '55', '56', '57', '58', '59',
       '60', '61A', '61B', '61C', '61D', '64', '67', '69', '71A', '71B', '71C',
       '71D', '75', '77', '79', '82', '83', '86', '87', '88', '89', '91',
       '28X', 'G2', 'P1', 'P68', 'Y47', 'Y49', '6', '81', '93', '11', '53',
       'Y46', '74', '53L'],
      dtype='object', name='Route')

In [ ]:
x.loc[x['51'] == 0].loc[x[51]==1.0] # Here is why, late at night, 51 is the only route
# with busses in service; since there are no routes like 'Y49' or 'P1' in service
# during that collection, the route was naturally stored as an int--not matching up with
# the '51', which was auto-converted to a string since there were string routes
# in that collection.
# The fix for this is seen in .apply(lambda x: str(x)) being added
# for the column title definition (routes)

KeyError: 51